In [1]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 
import pandas as pd
import numpy as np
import string, os 
import warnings

warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)


Artigo de Referência:

https://towardsdatascience.com/automatically-generate-hotel-descriptions-with-lstm-afa37002d4fc

https://github.com/susanli2016/NLP-with-Python/blob/master/Hotel%20Description%20Generation%20LSTM.ipynb


In [28]:
#hotel_df = pd.read_json(r"C:\Users\jtavares\Downloadsinventory_hotel_BR_pt.json")
hotel_df= pd.read_json(r'C:/Users/jgjoa/Desktop/FGV/inventory_hotel_BR_pt.json')
#all_descriptions = list(hotel_df['description'].desc.values)
#len(all_descriptions)

In [29]:
hotel_df = hotel_df[['name','description']]
hotel_df = hotel_df[0:500]
all_descriptions = list(hotel_df['description'].values)

hotel_df

,name,description
0,Best Western Hôtel New York Nice,"Located in Nice's city center, Best Western Ho..."
1,Comfort Inn Hyde Park,"In London's Bayswater district, the Comfort In..."
2,Best Western Plus Dayton Hotel & Suites,Breakfast is offered daily and rooms feature f...
3,"The Elizabeth Hotel, Autograph Collection","The Elizabeth Hotel, Autograph Collection feat..."
4,Grand Mercure Vila Olímpia,"Localizado no coração de São Paulo, o Grand Me..."
...,...,...
495,Kelta,"In the Hostería Kelta, located in the Province..."
496,WorldMark Running Y,"Offering an indoor swimming pool, this Klamath..."
497,Hotel Kirst,<h3>Localização Do Estabelecimento</h3>\nCom u...
498,Motel 6,A short drive from beautiful beaches and easil...


In [30]:
hotel_df.shape


(500, 2)

In [31]:
corpus = [x for x in all_descriptions]
corpus[:1]


["Located in Nice's city center, Best Western Hotel Lakmi Nice is located on Avenue Jean Medecin, Nice's main shopping street which leads to Place Massena. It is next to Nice Etoile Shopping Center and is just 1969 feet from Nice- Ville Train Station. Each spacious air-conditioned guest room has a flat- screen TV and a private bathroom fitted with a hairdryer. Some of the soundproofed rooms feature wrought iron furniture and some rooms offer a balcony. WiFi access is available throughout the hotel. A buffet breakfast is served every morning in the breakfast room at the Best Western Hotel Lakmi Nice. Newspapers are available for guests and there is also a bar on site. The Promenade des Anglais is a 13-minute walk from the hotel and Vieux Nice, the historic district, is a 15-minute walk away. Six private parking spaces are available on site at an additional cost, which cannot be booked in advance. ((* Sorry, this information is not available in the selected language and is shown in EN)."

In [32]:
t = Tokenizer(num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ', char_level=False, oov_token=None, document_count=0)
t.fit_on_texts(corpus)

In [33]:
# A dictionary of words and their counts.
print(t.word_counts)

# A dictionary of words and how many documents each appeared in.
print(t.word_docs)

# An integer count of the total number of documents that were used to fit the Tokenizer (i.e. total number of documents)
print(t.document_count)

# A dictionary of words and their uniquely assigned integers.
print(t.word_index)

OrderedDict([('located', 278), ('in', 1800), ("nice's", 2), ('city', 151), ('center', 403), ('best', 41), ('western', 34), ('hotel', 994), ('lakmi', 2), ('nice', 6), ('is', 2203), ('on', 285), ('avenue', 6), ('jean', 1), ('medecin', 1), ('main', 15), ('shopping', 73), ('street', 36), ('which', 64), ('leads', 1), ('to', 517), ('place', 44), ('massena', 1), ('it', 99), ('next', 10), ('etoile', 1), ('and', 2931), ('just', 119), ('1969', 4), ('feet', 70), ('from', 730), ('ville', 3), ('train', 27), ('station', 78), ('each', 130), ('spacious', 51), ('air', 135), ('conditioned', 79), ('guest', 113), ('room', 254), ('has', 137), ('a', 2585), ('flat', 148), ('screen', 151), ('tv', 220), ('private', 206), ('bathroom', 106), ('fitted', 25), ('with', 758), ('hairdryer', 51), ('some', 77), ('of', 853), ('the', 2940), ('soundproofed', 5), ('rooms', 505), ('feature', 75), ('wrought', 1), ('iron', 6), ('furniture', 12), ('offer', 63), ('balcony', 37), ('wifi', 109), ('access', 224), ('available', 718

In [34]:
print('Found %s unique tokens.' % len(t.word_index))

Found 7553 unique tokens.


In [21]:
# Tokenization
t = Tokenizer(num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ', char_level=False, oov_token=None, document_count=0)

def get_sequence_of_tokens(corpus):
    t.fit_on_texts(corpus)
    total_words = len(t.word_index) + 1
    
    input_sequences = []
    for line in corpus:
        token_list = t.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
            
    return input_sequences, total_words

In [50]:
input_sequences, total_words = get_sequence_of_tokens(corpus)


In [52]:
input_sequences[:10]

[[45, 5],
 [45, 5, 2698],
 [45, 5, 2698, 94],
 [45, 5, 2698, 94, 33],
 [45, 5, 2698, 94, 33, 279],
 [45, 5, 2698, 94, 33, 279, 343],
 [45, 5, 2698, 94, 33, 279, 343, 7],
 [45, 5, 2698, 94, 33, 279, 343, 7, 2699],
 [45, 5, 2698, 94, 33, 279, 343, 7, 2699, 1337],
 [45, 5, 2698, 94, 33, 279, 343, 7, 2699, 1337, 4]]

In [53]:
total_words

7554

In [54]:
# pad sequences 
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen = max_sequence_len, padding = 'pre'))
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes = total_words)
    
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(input_sequences)

In [55]:
def create_model(max_sequence_len, total_words):
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=max_sequence_len - 1))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

model = create_model(max_sequence_len, total_words)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 727, 10)           75540     
                                                                 
 lstm (LSTM)                 (None, 100)               44400     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 7554)              762954    
                                                                 
Total params: 882,894
Trainable params: 882,894
Non-trainable params: 0
_________________________________________________________________


In [56]:
model.fit(predictors, label, epochs=10, verbose=5)

Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10


In [61]:
def generate_text(seed_text, next_words, model, max_seq_len):
    for _ in range(next_words):
        token_list = t.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_seq_len-1, padding='pre')
        
        #predicted = model.predict_classes(token_list, verbose=0)
        predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)
        output_word = ''
        
        for word,index in t.word_index.items():
            if index == predicted:
                output_word = word
                break
                
        seed_text = seed_text + " " + output_word
        
    return seed_text.title()

In [62]:
print(generate_text("hilton seattle downtown", 100, model, max_sequence_len))


Hilton Seattle Downtown Hotel Is Located In The Heart Of The Historic Of The Hotel In The Heart Of The Heart Of The Heart Of The Hotel The Hotel Is Within Walking Distance Of The Hotel And The Beach Of The Hotel The Hotel Is Within Walking Distance Of The Hotel And The Hotel Is Just Minutes Away From The Hotel And The Hotel Is A Short Drive Away The Hotel Is A Short Drive Away And A Variety Of The Hotel The Hotel Offers A Variety Of The Hotel In The Hotel In The Hotel In The Heart Of The Heart Of


In [64]:
print(generate_text("Holiday Inn Express - New York City, City-center hotel, walk to Madison Square Garden", 100, model, max_sequence_len))

Holiday Inn Express - New York City, City-Center Hotel, Walk To Madison Square Garden By Wyndham Wapakoneta Is A Short Drive From The Beach Of The Beach The Hotel Is Just A Short Drive From The Hotel And The Beach Of The City Of The Hotel The Hotel Is Just Close From The Historic Of The The Conditioned And The Hotel The Hotel Is Just Minutes From The Historic Of The Hotel The Hotel Is Within Walking Distance Of The Hotel The Hotel Is Just A Short Drive Away And A Variety Of The Hotel The Hotel Offers A Variety Of The Hotel In The Hotel In The Hotel In The Hotel In The


# Teste 2 - Apenas Hoteis Brasileiros (Descrição em português)

In [3]:
hotelBR_df= pd.read_json(r'C:/Users/jgjoa/Desktop/FGV/inventory_hotel_BR_pt.json')
hotelBR_df = hotelBR_df[hotelBR_df['country']=='Brasil']


In [29]:
hotelBR_df = hotelBR_df[0:500]
all_descriptionsBR = list(hotelBR_df['description'].values)

hotelBR_df.describe()

,id_offer,stars,price,price_before_discount,price_cents
count,5.000000e+02,500.000000,5.000000e+02,5.000000e+02,500.000000
mean,8.779623e+05,3.040000,4.548309e+04,4.548309e+04,454.830880
std,2.671592e+05,0.831735,1.212900e+05,1.212900e+05,1212.899616
min,2.475300e+04,1.000000,4.500000e+03,4.500000e+03,45.000000
25%,7.623340e+05,3.000000,1.544450e+04,1.544450e+04,154.445000
50%,8.380850e+05,3.000000,2.225600e+04,2.225600e+04,222.560000
75%,1.065782e+06,3.000000,3.612250e+04,3.612250e+04,361.225000
max,1.430929e+06,5.000000,1.099890e+06,1.099890e+06,10998.900000


In [31]:
hotelBR_df.columns

Index(['sku', 'possible_integrators', 'pos_disabled', 'name', 'id_offer',
       'description', 'external_reference', 'search_engine_id', 'stars',
       'room', 'amenity', 'tag', 'small_description', 'zipcode', 'country',
       'country_acronym', 'state', 'state_acronym', 'region', 'city', 'street',
       'neighborhood', 'contact_phone', 'coords', 'price_currency', 'price',
       'price_before_discount', 'product_type', 'short_description', 'gallery',
       'image', 'app_url', 'price_cents', 'url', 'search_url'],
      dtype='object')

In [49]:
hotelBR_df[['name','price_currency','possible_integrators','description','short_description','small_description']]

,name,price_currency,possible_integrators,description,short_description,small_description
4,Grand Mercure Vila Olímpia,BRL,[OMN-4454],"Localizado no coração de São Paulo, o Grand Me...","Localizado no coração de São Paulo, o Grand Me...","Localizado no coração de São Paulo, o Grand Me..."
17,Porto da Ilha Hotel,BRL,"[OMN-1103, JNP-JP049872]",Hotel Porto da Ilha offers stylish rooms in th...,Não perca as várias atividades recreativas e d...,Não perca as várias atividades recreativas e d...
37,Icamiabas Parintins,BRL,[OMN-9046],Você é nosso convidado para conhecer o Hotel I...,Você é nosso convidado para conhecer o Hotel I...,Você é nosso convidado para conhecer o Hotel I...
50,Pousada Raio De Sol,BRL,[OMN-15879],"Situada em Barreirinhas, a 2,1 km do cais e a ...","Situada em Barreirinhas, a 2,1 km do cais e a ...","Situada em Barreirinhas, a 2,1 km do cais e a ..."
98,Hotel Santa Fé - Goiânia,BRL,[JNP-JP657426],<h3>Localização Do Estabelecimento</h3>\nCom u...,Com uma estadia Hotel Santa Fé em Goiânia (Nor...,Com uma estadia Hotel Santa Fé em Goiânia (Nor...
...,...,...,...,...,...,...
8779,Hotel São Jorge,BRL,[JNP-JP126854],Property Location With a stay at Hotel São Jor...,Property Location With a stay at Hotel São Jor...,Property Location With a stay at Hotel São Jor...
8791,Ibis budget RJ Nova América,BRL,[OMN-8660],"Ibis budget RJ Nova América, um hotel econômic...","Ibis budget RJ Nova América, um hotel econômic...","Ibis budget RJ Nova América, um hotel econômic..."
8800,Odage Hotel,BRL,[JNP-JP609301],Property Location A stay at Odage Hotel places...,Property Location A stay at Odage Hotel places...,Property Location A stay at Odage Hotel places...
8809,Hotel Imperial,BRL,[JNP-JP316814],"Aproveite comodidades convenientes, como Wi-Fi...",Algumas das comodidades e serviços em destaque...,Algumas das comodidades e serviços em destaque...


In [5]:
corpusBR = [x for x in all_descriptionsBR]
corpusBR[:1]


['Localizado no coração de São Paulo, o Grand Mercure SP Vila Olimpia (ex Caesar Business) está próximo a grandes centros financeiros e importantes empresas. Por perto, há também excelentes bares e restaurantes, shoppings e variados locais de entreteni']

In [6]:
tBR = Tokenizer(num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ', char_level=False, oov_token=None, document_count=0)
tBR.fit_on_texts(corpusBR)

In [7]:
# A dictionary of words and their counts.
print(tBR.word_counts)

# A dictionary of words and how many documents each appeared in.
print(tBR.word_docs)

# An integer count of the total number of documents that were used to fit the Tokenizer (i.e. total number of documents)
print(tBR.document_count)

# A dictionary of words and their uniquely assigned integers.
print(tBR.word_index)

OrderedDict([('localizado', 70), ('no', 114), ('coração', 7), ('de', 723), ('são', 57), ('paulo', 32), ('o', 300), ('grand', 3), ('mercure', 8), ('sp', 4), ('vila', 13), ('olimpia', 1), ('ex', 1), ('caesar', 1), ('business', 7), ('está', 87), ('próximo', 32), ('a', 598), ('grandes', 1), ('centros', 4), ('financeiros', 1), ('e', 597), ('importantes', 2), ('empresas', 1), ('por', 33), ('perto', 3), ('há', 8), ('também', 25), ('excelentes', 3), ('bares', 12), ('restaurantes', 18), ('shoppings', 8), ('variados', 1), ('locais', 1), ('entreteni', 1), ('não', 23), ('perca', 12), ('as', 88), ('várias', 48), ('atividades', 24), ('recreativas', 23), ('entretimento', 9), ('ao', 101), ('seu', 43), ('dispor', 34), ('incluindo', 49), ('uma', 187), ('sala', 38), ('fitness', 22), ('espaço', 59), ('oferece', 88), ('ainda', 19), ('wi', 123), ('fi', 122), ('grátis', 76), ('um', 146), ('televisor', 32), ('comum', 30), ('entre', 23), ('facilidades', 24), ('adicionais', 26), ('contam', 14), ('se', 54), ('ap

In [8]:
print('Found %s unique tokens.' % len(tBR.word_index))

Found 3279 unique tokens.


In [9]:
# Tokenization
tBR = Tokenizer(num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ', char_level=False, oov_token=None, document_count=0)

def get_sequence_of_tokensBR(corpusBR):
    tBR.fit_on_texts(corpusBR)
    total_wordsBR = len(tBR.word_index) + 1
    
    input_sequencesBR = []
    for line in corpusBR:
        token_list = tBR.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequencesBR.append(n_gram_sequence)
            
    return input_sequencesBR, total_wordsBR

In [10]:
input_sequencesBR, total_wordsBR = get_sequence_of_tokensBR(corpusBR)


In [11]:
input_sequencesBR[:10]

[[42, 23],
 [42, 23, 368],
 [42, 23, 368, 1],
 [42, 23, 368, 1, 53],
 [42, 23, 368, 1, 53, 92],
 [42, 23, 368, 1, 53, 92, 5],
 [42, 23, 368, 1, 53, 92, 5, 671],
 [42, 23, 368, 1, 53, 92, 5, 671, 333],
 [42, 23, 368, 1, 53, 92, 5, 671, 333, 549],
 [42, 23, 368, 1, 53, 92, 5, 671, 333, 549, 218]]

In [12]:
total_wordsBR

3280

In [13]:
# pad sequences 
def generate_padded_sequencesBR(input_sequencesBR):
    max_sequence_lenBR = max([len(x) for x in input_sequencesBR])
    input_sequencesBR = np.array(pad_sequences(input_sequencesBR, maxlen = max_sequence_lenBR, padding = 'pre'))
    predictorsBR, labelBR = input_sequencesBR[:,:-1],input_sequencesBR[:,-1]
    labelBR = ku.to_categorical(labelBR, num_classes = total_wordsBR)
    
    return predictorsBR, labelBR, max_sequence_lenBR

predictorsBR, labelBR, max_sequence_lenBR = generate_padded_sequencesBR(input_sequencesBR)

In [14]:
def create_modelBR(max_sequence_lenBR, total_wordsBR):
    modelBR = Sequential()
    
    # Add Input Embedding Layer
    modelBR.add(Embedding(total_wordsBR, 10, input_length=max_sequence_lenBR - 1))
    
    # Add Hidden Layer 1 - LSTM Layer
    modelBR.add(LSTM(100))
    modelBR.add(Dropout(0.1))
    
    # Add Output Layer
    modelBR.add(Dense(total_wordsBR, activation='softmax'))

    modelBR.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return modelBR

modelBR = create_modelBR(max_sequence_lenBR, total_wordsBR)
modelBR.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 59, 10)            32800     
                                                                 
 lstm (LSTM)                 (None, 100)               44400     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 3280)              331280    
                                                                 
Total params: 408,480
Trainable params: 408,480
Non-trainable params: 0
_________________________________________________________________


In [16]:
modelBR.fit(predictorsBR, labelBR, epochs=100, verbose=1)

Epoch 1/100
583/583 [==============================] - 55s 95ms/step - loss: 6.2880
Epoch 2/100
583/583 [==============================] - 56s 95ms/step - loss: 5.9250
Epoch 3/100
583/583 [==============================] - 71s 121ms/step - loss: 5.4902
Epoch 4/100
583/583 [==============================] - 63s 108ms/step - loss: 5.1073
Epoch 5/100
583/583 [==============================] - 65s 111ms/step - loss: 4.8011
Epoch 6/100
583/583 [==============================] - 60s 102ms/step - loss: 4.5523
Epoch 7/100
583/583 [==============================] - 58s 100ms/step - loss: 4.3383
Epoch 8/100
583/583 [==============================] - 63s 108ms/step - loss: 4.1424
Epoch 9/100
583/583 [==============================] - 59s 101ms/step - loss: 4.0517
Epoch 10/100
583/583 [==============================] - 58s 100ms/step - loss: 3.8610
Epoch 11/100
583/583 [==============================] - 59s 101ms/step - loss: 3.7229
Epoch 12/100
583/583 [==============================] - 51s 88ms/

In [21]:
def generate_textBR(seed_textBR, next_wordsBR, modelBR, max_seq_lenBR):
    for _ in range(next_wordsBR):
        token_listBR = tBR.texts_to_sequences([seed_textBR])[0]
        token_listBR = pad_sequences([token_listBR], maxlen=max_seq_lenBR-1, padding='pre')
        
        #predicted = model.predict_classes(token_list, verbose=0)
        predicted = np.argmax(modelBR.predict(token_listBR, verbose=0), axis=-1)
        output_word = ''
        
        for word,index in tBR.word_index.items():
            if index == predicted:
                output_word = word
                break
                
        seed_textBR = seed_textBR + " " + output_word
        
    return seed_textBR.title()

In [28]:
print(generate_textBR("Buzios Beach localizado em Buzios", 100, modelBR, max_sequence_lenBR))

Um Buffet De Café Da Manhã From Coqueiros Para Espaço E In Salvador Grande Das Boa Viagem Park This Pousada Is 30 5 Mi 2 6 Km From Quadrado Church And Teatro 11 Center Rooms 48 3 Mi 4 7 Km Rooms Make Com 3 Condicionado Você 4 E F Service Km De Barto Privativo No Local Por Um Restaurante São Horas 
